In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
data_inf = pd.read_csv('inf_data.csv')
data_inf

,Unnamed: 0,review_id,reviewer,movie,rating,review_summary,review_date,spoiler_tag,review_detail,helpful
0,340291,rw0704975,nbarratt,The Matrix Reloaded (2003),5.0,It's been a long time coming...,15 July 2003,0,Trinity falling backwards through the window i...,"['0', '0']"
1,195744,rw0286124,the red duchess,Close-Up (1990),8.0,Almost lives up to his reputation.,8 November 2000,0,This is apparently the most important film of ...,"['9', '18']"
2,274406,rw0631834,avanhame,The Patriot (2000),10.0,Excellent,22 July 2000,0,"After reading the reviews, I felt it necessary...","['13', '20']"
3,83292,rw0156557,dean29,Suspiria (1977),2.0,This movie is not scary,17 August 2002,0,Warning: If you have heard good reviews about...,"['10', '24']"
4,370401,rw0738431,MattPDX,The Trip (2002),10.0,GREAT movie!!!,26 February 2002,0,Miles Swain has directed an excellent movie. T...,"['1', '3']"
5,430648,rw0805812,cscorder,Sakuya: yôkaiden (2000),7.0,Nice Eastern eye candy,8 November 2001,0,Credible action flick even if it's obviously a...,"['1', '1']"
6,123423,rw0202834,isotope2112,The Meaning of Life (1983),5.0,"Funny, but uneven",21 March 2003,0,I just saw The Meaning of Life again recently....,"['0', '5']"
7,296324,rw0656047,FlickJunkie-2,The Omega Code (1999),2.0,Uniformly awful,27 April 2000,0,This film was dreadful in almost every regard....,"['3', '7']"
8,318951,rw0681195,Zar,Maciste il vendicatore dei Maya (1965),3.0,A very weird experience!,6 June 2001,0,The extremely obscure Kirk Morris vehicle MACI...,"['4', '4']"
9,343408,rw0708500,azaro666,The Tailor of Panama (2001),5.0,What did I miss?,25 May 2001,0,"I am still convinced, even now, that I have mi...","['0', '0']"


In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
import string
import requests
import tensorflow as tf

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ikhbar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ikhbar\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ikhbar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [36]:
tf.__version__

'2.9.2'

In [58]:
with open('frontend/model/preprocess_text.pkl', 'rb') as f:
    procc = pickle.load(f)

c:\Users\ikhbar\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\ikhbar\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [13]:
stop_words = list(STOPWORDS)
stemmer = PorterStemmer()
lemma = WordNetLemmatizer()

In [14]:
def full_remove(x, removal_list): 
    for w in removal_list:
        x = x.replace(w, ' ')
    return x

stop_words.extend(['s','ve','t','nt','m','ll']) #Menambahkan stop words list

def text_processing_2(text):
  text = text.lower() #Set lowercase pada kata
  text = re.sub(r"http\S+", " ", text) #Menghilangkan link
  text = re.sub(r"www.\S+", " ", text) #Menghilangkan link
  text = re.sub(r"\n"," ",text) #Menghilangkan \n
  text = ' '.join([full_remove(x, list(string.punctuation)) for x in text.split()]) #Menghilangkan punctuation
  text = text.strip() #Menghilangkan spasi kosong ataupun tab pada awal dan akhir kalimat
  text = re.sub("[^A-Za-z\s']"," ", text) #Menghilangkan yang bukan huruf
  tokens = word_tokenize(text) # Mendeteksi token pada kalimat
  text = [lemma.lemmatize(word) for word in tokens if word not in stop_words]
  text = ' '.join(text)
  return text    

In [15]:
data_inf['proc'] = data_inf['review_detail'].apply(lambda x: text_processing_2(x))

In [103]:
X_inf = np.array([data_inf['proc'].iloc[5]])

In [104]:
X_inf

array(['credible action flick obviously kid movie great special effect primary attraction cast treat material deadly seriousness fun watch fight scene average part looked like bad rehash h r puffenstuff'],
      dtype='<U194')

In [105]:
contoh_inp = procc.transform(X_inf).toarray()

In [106]:
input_data_json = json.dumps({
    "signature_name": "serving_default",
    "instances": contoh_inp.tolist()
})

In [107]:
URL = "http://backend-ikhbar-fin.herokuapp.com/v1/models/review_model:predict"
r = requests.post(URL, data=input_data_json)

print(r.status_code)
# berikut bentuk data hasil prediksi
res = r.json()
res

200


{'predictions': [[0.236282468, 0.579923928, 0.183793634]]}

In [81]:
np.argmax(res['predictions'][0])

0

In [76]:
import sklearn

In [77]:
sklearn.__version__

'1.1.2'